In [13]:
import pandas as pd
# import numpy as np
import pickle
# import seaborn as sns
from random import choice
import re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

In [2]:
def decode_lyrics(s):
    s = s.encode('ascii', 'ignore')
    s = s.decode()
    s = s.replace('\n', ' ')
    s = s.replace('-', ' ')
    s = re.sub(r'[\[].*?[\)\]]', ' ', s)
    
    return s

def recommended_albums(sample_album):
    
    album_index = mapping[sample_album]
    similarity_score = list(enumerate(similarity_matrix[album_index]))
    similarity_score = sorted(similarity_score, key=lambda x: x[1], reverse=True)
    
    # get score of 10 most similar movies. Ignore first movie.
    similarity_score = similarity_score[1:10]
    
    # return movie names using the mapping series
    
    album_indices = [i[0] for i in similarity_score]
    
    return (album_df['album_release'].iloc[album_indices])
#     return similarity_score

In [3]:
dfs = pickle.load(open('dfs_w_lyrics.pkl', 'rb'))
tracks = pd.concat(dfs)
tracks.reset_index(inplace=True)
lead_artist = []
feat_artists = []

for artists in tracks.artist:
    lead_artist.append(artists[0])
    feat_artists.append(artists[1:])

tracks['lead_artist'] = lead_artist
tracks['feat_artists'] = feat_artists
tracks['album_release'] = tracks.album + ' by ' + tracks.lead_artist

In [4]:
lyrics_flag = pd.isna(tracks['lyrics'])
lyrical_db = tracks[~lyrics_flag]
lyrical_db = lyrical_db[['album_release', 'lyrics']]

In [5]:
lyrical_db

,album_release,lyrics
0,After Hours by The Weeknd,[Verse]\nTake off my disguise\nI'm living some...
1,After Hours by The Weeknd,"[Intro]\nNo-no, no-no, no-no-no\n\n[Verse 1]\n..."
2,After Hours by The Weeknd,"[Intro]\nNo, yeah\nOh, yeah\nOh, yeah\nYeah, u..."
3,After Hours by The Weeknd,"[Verse 1]\nWhen I saw the signs, I shoulda let..."
4,After Hours by The Weeknd,[Verse 1]\nI used to pray when I was sixteen\n...
...,...,...
6939,zero_one by The Living Tombstone,"[Verse 1]\nAnother little abuse, another point..."
6940,zero_one by The Living Tombstone,[Intro]\n(Mama told me this is the way we talk...
6941,zero_one by The Living Tombstone,[Intro]\n(Animal)\n(Animal Animal Animal)\n(An...
6942,zero_one by The Living Tombstone,"[Verse 1]\nPack your bag, hop in a car\nAnothe..."


In [6]:
lyrics_by_album = lyrical_db.groupby('album_release')['lyrics'].apply(lambda x: ' '.join(x))
lyrics_by_album = lyrics_by_album.apply(decode_lyrics)

lyrics = lyrical_db['lyrics']
_ = lyrics_by_album.reset_index()

In [11]:
# lyrics_by_album = lyrics_by_album.apply(decode_lyrics)

In [12]:
# lyrics_by_album = lyrical_db.groupby('album_and_artist')['lyrics'].apply(lambda x: ' '.join(x))

In [13]:
# lyrics_by_album = lyrics_by_album.apply(decode_lyrics)

In [7]:
tfidf = TfidfVectorizer(stop_words='english')
lyric_vec = tfidf.fit_transform(lyrics_by_album)
lyric_vec.shape;

In [8]:
print(f'{tracks.album.nunique()} albums.')
print(f'{lyric_vec.shape[1]} unique single n-grams in album corpus.')

508 albums.
59938 unique single n-grams in album corpus.


In [9]:
similarity_matrix = linear_kernel(lyric_vec, lyric_vec)

In [10]:
mapping = pd.Series(_.index, index=_.album_release)

In [11]:
album_df = lyrics_by_album.reset_index()

### List of albums

In [12]:
for album in lyrical_db.album_release.unique():
    print(album)

After Hours by The Weeknd
ASTROWORLD by Travis Scott
American Teen by Khalid
AI YoungBoy 2 by YoungBoy Never Broke Again
A Love Letter To You 4 by Trippie Redd
Artist 2.0 by A Boogie Wit da Hoodie
ANTI (Deluxe) by Rihanna
A Love Letter To You 3 by Trippie Redd
A Letter To My Younger Self by Quinn XCII
AM by Arctic Monkeys
Appetite For Destruction by Guns N' Roses
Artist 2.0 (Deluxe) by A Boogie Wit da Hoodie
AT.LONG.LAST.A$AP by A$AP Rocky
A Fever You Can't Sweat Out by Panic! At The Disco
American Beauty/American Psycho by Fall Out Boy
A Love Letter To You by Trippie Redd
Artist by A Boogie Wit da Hoodie
beerbongs & bentleys by Post Malone
Blonde by Frank Ocean
BALLADS 1 by Joji
B4 The Storm by Internet Money
Birds In The Trap Sing McKnight by Travis Scott
Birds In The Trap Sing McKnight by Young Thug
Beauty Behind The Madness by The Weeknd
Blurryface by Twenty One Pilots
Be Like That (feat. Swae Lee & Khalid) by Kane Brown
Baby On Baby by DaBaby
Blueberry Faygo by Lil Mosey
Breaking 

In [29]:
# choice(lyrical_db.album_release.unique())
print(f'Recommending albums from {choice(lyrical_db.album_release.unique())}')
recommended_albums(choice(lyrical_db.album_release.unique()))

Recommending albums from X (feat. Maluma & Ozuna) [Remix] by Nicky Jam


319                       Stoney (Deluxe) by Post Malone
438    Young The Giant (Special Edition) by Young the...
286                       Queen Of The Clouds by Tove Lo
351                     Ugly is Beautiful by Oliver Tree
369                            Ventura by Anderson .Paak
154                                Indigo by Chris Brown
445                         ZERO : FEVER Part.1 by ATEEZ
210                                Lover by Taylor Swift
297                 Red (Deluxe Edition) by Taylor Swift
Name: album_release, dtype: object

### Manually test albums

In [31]:
# manually test album
recommended_albums('Oracular Spectacular by MGMT')

397                      Wiped Out! by The Neighbourhood
343                        There's Really A Wolf by Russ
342                                     The Search by NF
429                                     YOU by Ali Gatie
175                       Justified by Justin Timberlake
213    Made In The A.M. (Deluxe Edition) by One Direc...
268                                     Perception by NF
35                                       COSMIC by Bazzi
312                                    Scorpion by Drake
Name: album_release, dtype: object